In [26]:
%load_ext kedro

The kedro extension is already loaded. To reload it, use:
  %reload_ext kedro


In [27]:
df = catalog.load('raw_data')
df.info()

[10/14/25 22:51:47] INFO     Loading data from raw_data (ParquetDataset)...                    ]8;id=952228;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=347625;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/io/data_catalog.py#1046\1046]8;;\

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55043 entries, 0 to 55042
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        55043 non-null  object
 1   category    55043 non-null  object
 2   item        55043 non-null  object
 3   unit        55043 non-null  object
 4   low_price   55043 non-null  object
 5   high_price  55043 non-null  object
dtypes: object(6)
memory usage: 2.5+ MB


In [28]:
import pandas as pd

In [29]:
df['date'] = pd.to_datetime(df['date'])

In [30]:

numeric_cols = ['low_price', 'high_price']

for col in numeric_cols:
    print(f"\n=== CHECKING {col} ===")
    
    numeric_series = pd.to_numeric(df[col], errors='coerce')
    non_numeric_mask = numeric_series.isna() & df[col].notna()
    
    if non_numeric_mask.any():
        print(f"Non-numeric values found in {col}:")
        non_numeric_values = df.loc[non_numeric_mask, col].unique()
        print(non_numeric_values)
        print(f"Count: {non_numeric_mask.sum()}")
    else:
        print(f"All values in {col} are numeric!")



=== CHECKING low_price ===
Non-numeric values found in low_price:
['বাজারে পাওয়া']
Count: 1

=== CHECKING high_price ===
Non-numeric values found in high_price:
['যায়নি']
Count: 1


In [31]:
df.loc[df['low_price'] == 'বাজারে পাওয়া', 'low_price']


42669    বাজারে পাওয়া
Name: low_price, dtype: object

In [32]:
df.loc[df['high_price'] == 'যায়নি', 'high_price']


42669    যায়নি
Name: high_price, dtype: object

**there seems to be one row with non numeric column. possible typo**

In [33]:
df['low_price'] = pd.to_numeric(df['low_price'], errors='coerce')
df['high_price'] = pd.to_numeric(df['high_price'], errors='coerce')

In [34]:
df['unit'].value_counts()


unit
প্রতি কেজি          38330
১ কেজি               4144
প্রতি লিটার          3106
প্রতি কেজি প্যাঃ     2072
প্রতি মেঃটন          2072
৫ লিটার              1502
১ লিটার              1368
প্রতি দিস্তা         1036
প্রতি হালি           1036
2 লিটার               375
50.00-52.00             2
Name: count, dtype: int64

In [35]:
df[df['unit'] == '50.00-52.00']

,date,category,item,unit,low_price,high_price
5664,2023-11-13,মসলাঃ,তেজপাতা,50.00-52.00,150.0,200.0
41850,2023-11-15,মসলাঃ,তেজপাতা,50.00-52.00,150.0,200.0


**some units seem to have been mislabeled. ahve to fill them with proper unit for the item.**

In [36]:
df[df['item'] == 'তেজপাতা']['unit']


36       প্রতি কেজি
91       প্রতি কেজি
146      প্রতি কেজি
198      প্রতি কেজি
255      প্রতি কেজি
            ...    
54811    প্রতি কেজি
54865    প্রতি কেজি
54917    প্রতি কেজি
54971    প্রতি কেজি
55025    প্রতি কেজি
Name: unit, Length: 1036, dtype: object

In [37]:
df[df['unit'] == '50.00-52.00']['unit'] = 'প্রতি কেজি'

[10/14/25 22:52:00] WARNING  /tmp/ipykernel_18497/905726389.py:1: SettingWithCopyWarning:           ]8;id=397369;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=45584;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/warnings.py#110\110]8;;\
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df[df['unit'] == '50.00-52.00']['unit'] = 'প্রতি কেজি'                                  
                                                                                                                   

In [ ]:
unit_mapping = {
    'প্রতি কেজি': '1 KG',
    '১ কেজি': '1 KG', 
    'প্রতি লিটার': '1 Liter',
    'প্রতি কেজি প্যাঃ': '1 KG',
    'প্রতি মেঃটন': '1 Metric Tonne',
    '৫ লিটার': '5 Liter',
    '১ লিটার': '1 Liter',
    'প্রতি দিস্তা': '25 Pieces',
    'প্রতি হালি': '4 Pieces',
    '2 লিটার': '2 Liter'
}

df['unit'] = df['unit'].map(unit_mapping)